#### Calculate total GW and km needed for grid in NS by cases

In [5]:
import os
import sys

parent_dir = os.path.abspath('..')
if parent_dir not in sys.path:
    sys.path.append(parent_dir)

import pandas as pd
from constants import OFFSHORE_AREAS

#Haakon: for kabler legg til ["Cables/CablesN10", "Cables/CablesN20", "Cables/CablesN30", "Cables/CablesP10", "Cables/CablesP20", "Cables/CablesP30"]
CASES = ["Base", "NOgrid", "NoHubs", "NoHubsNOgrid", "OnlyCentral", "OnlyEU", "OnlyNorth"]
DF_BY_CASES = dict({})

Cable length

In [7]:
cable_length = pd.read_csv("../../EMPIRE_extension/Output/directional_lines.csv")
cable_length["FromNode"] = cable_length["FromNode"].apply(lambda n: n.replace(" ", ""))
cable_length["ToNode"] = cable_length["ToNode"].apply(lambda n: n.replace(" ", ""))
cable_length

,FromNode,ToNode,lineLength in km,LineType
0,MorayFirth,FirthofForth,215.667240,HVDC_Cable
1,MorayFirth,DoggerBank,509.370695,HVDC_Cable
2,FirthofForth,DoggerBank,333.632410,HVDC_Cable
3,MorayFirth,Hornsea,586.957998,HVDC_Cable
4,FirthofForth,Hornsea,387.595780,HVDC_Cable
...,...,...,...,...
388,EnergyhubCentral,GreatBrit.,491.825549,HVDC_OffshoreToOnshore
389,EnergyhubCentral,Denmark,376.970459,HVDC_OffshoreToOnshore
390,EnergyhubCentral,Netherlands,470.165660,HVDC_OffshoreToOnshore
391,EnergyhubCentral,Belgium,614.084832,HVDC_OffshoreToOnshore


In [8]:
for case in CASES:
    _df = pd.read_csv(f'../Results/{case}/results_output_transmission.csv')
    _df = _df[(_df["BetweenNode"].isin(OFFSHORE_AREAS)) | (_df["AndNode"].isin(OFFSHORE_AREAS))]
    _df = _df.rename(columns={"BetweenNode": "FromNode", "AndNode": "ToNode"})
    _df = _df.merge(cable_length, on=["FromNode", "ToNode"], how="left")
    _df["Transmission MW km"] = _df["transmissionInstalledCap_MW"] * _df["lineLength in km"]
    DF_BY_CASES[case] = _df

DF_BY_CASES

{'Base':               FromNode        ToNode     Period  transmissionInvCap_MW  \
 0           MorayFirth  FirthofForth  2020-2025           4.017357e-04   
 1           MorayFirth  FirthofForth  2025-2030           5.922289e-07   
 2           MorayFirth  FirthofForth  2030-2035           1.164921e-03   
 3           MorayFirth  FirthofForth  2035-2040           1.667952e+01   
 4           MorayFirth  FirthofForth  2040-2045           1.381993e+02   
 ...                ...           ...        ...                    ...   
 3139  EnergyhubCentral       Germany  2035-2040           2.067610e+03   
 3140  EnergyhubCentral       Germany  2040-2045           5.837300e+03   
 3141  EnergyhubCentral       Germany  2045-2050           8.168703e-04   
 3142  EnergyhubCentral       Germany  2050-2055           3.478651e-07   
 3143  EnergyhubCentral       Germany  2055-2060           2.154247e-05   
 
       transmissionInstalledCap_MW  DiscountedInvestmentCost_EuroPerMW  \
 0              

Filter by case and period

In [10]:
FILTER_CASE = "Base"
FILTER_PERIOD = "2045-2050"
MAX_CABLE_CAP = 2 # 1 or 2 GW

def calculate_cable_length_needed(df, cable_cap, period):
    total_length = 0
    for _, df_row in df.iterrows():
        num_cables = (df_row["transmissionInstalledCap_MW"] / 1000) // cable_cap + 1
        total_length += num_cables * df_row["lineLength in km"]
    start_year = 2023
    end_year = int(period.split("-")[1])
    num_years = end_year-start_year
    print(f"Total cable length needed until {end_year}: {round(total_length)} km with max {cable_cap}GW per cable")
    print(f"Average cable km installed per year until {end_year}: {round(total_length / num_years, 1)} km/yr")

filtered_df = DF_BY_CASES[FILTER_CASE]

# Threshold value 1 MW
filtered_df = filtered_df[(filtered_df["Period"] == FILTER_PERIOD) & (filtered_df["transmissionInstalledCap_MW"] > 1)]\
                            .sort_values("transmissionInstalledCap_MW", ascending=False)
print(f"Total cable capacitiy: {round(filtered_df["transmissionInstalledCap_MW"].sum() / 1000)} GW")
print(f"Total transmission in GW km: {round(filtered_df["Transmission MW km"].sum() / 1000)} GW km")
print(f"Average cable capacity between areas: {round(filtered_df["transmissionInstalledCap_MW"].mean() / 1000, 1)} GW\n")
calculate_cable_length_needed(filtered_df, MAX_CABLE_CAP, FILTER_PERIOD)
filtered_df.head(30)

Total cable capacitiy: 409 GW
Total transmission in GW km: 170017 GW km
Average cable capacity between areas: 2.7 GW

Total cable length needed until 2050: 117471 km with max 2GW per cable
Average cable km installed per year until 2050: 4350.8 km/yr


,FromNode,ToNode,Period,transmissionInvCap_MW,transmissionInstalledCap_MW,DiscountedInvestmentCost_EuroPerMW,transmissionExpectedAnnualVolume_GWh,ExpectedAnnualLosses_GWh,lineLength in km,LineType,Transmission MW km
3045,EnergyhubEU,Germany,2045-2050,5.687694e-04,29999.999936,4.400784e+01,182718.137908,5481.544137,565.794411,HVDC_OffshoreToOnshore,1.697383e+07
3141,EnergyhubCentral,Germany,2045-2050,8.168703e-04,29999.999930,8.385048e+01,181627.906753,5448.837203,750.895383,HVDC_OffshoreToOnshore,2.252686e+07
3093,EnergyhubNorth,Germany,2045-2050,1.202885e+03,29999.999879,1.691766e+08,236848.194850,7105.445846,1029.146418,HVDC_OffshoreToOnshore,3.087439e+07
3109,EnergyhubCentral,GreatBrit.,2045-2050,3.516204e-06,25451.452841,2.365480e-01,111634.544374,3349.036331,491.825549,HVDC_OffshoreToOnshore,1.251767e+07
3013,EnergyhubEU,GreatBrit.,2045-2050,2.302270e-06,11362.548204,1.708127e-01,56872.472373,1706.174171,542.500872,HVDC_OffshoreToOnshore,6.164192e+06
3021,EnergyhubEU,Denmark,2045-2050,1.203419e-06,9051.686905,4.283521e-02,43039.867397,1291.196022,259.821659,HVDC_OffshoreToOnshore,2.351824e+06
2829,HelgoländerBucht,Germany,2045-2050,0.000000e+00,6566.000000,0.000000e+00,37426.132164,1122.783965,439.059020,HVDC_OffshoreToOnshore,2.882862e+06
3053,EnergyhubNorth,NO2,2045-2050,1.892968e-06,6016.552295,9.203232e-02,38310.628655,1149.318860,355.199524,HVDC_OffshoreToOnshore,2.137077e+06
3037,EnergyhubEU,Belgium,2045-2050,3.305212e+03,5972.498704,2.265142e+08,30148.907222,904.467217,501.044249,HVDC_OffshoreToOnshore,2.992486e+06
3061,EnergyhubNorth,GreatBrit.,2045-2050,1.339667e+03,5404.110104,1.248024e+08,33473.050055,1004.191502,681.400749,HVDC_OffshoreToOnshore,3.682365e+06
